In [2]:
import pandas as pd
import numpy as np
import random
import os
import sys
sys.path.append('../..')
from modules.many_features import utils, constants, env
from modules.many_features.random_agent import RandomAgent
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [4]:
#df = pd.read_csv('../../data/anemia_synth_dataset_some_nans_unspecified_more_feats.csv')
#df = pd.read_csv('../../data/more_features/more_feats_new_labels_0.1.csv')
df = pd.read_csv('../../data/more_features/more_feats_correlated_noisy_2_missing_05.csv')
df = df.fillna(-1)
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,6.863691,341.823457,-1.000000,0.681783,284.835163,92.819484,-1.000000,2.218400,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,20.591074,-1.000000,Inconclusive diagnosis
1,8.745950,104.470248,0.461482,0.787183,409.993509,104.531457,77.222298,2.510043,1,0.713849,-1.000000,51.527756,7.722555,7.822857,-1.0,26.237850,18.835005,Vitamin B12/Folate deficiency anemia
2,10.308881,475.936322,-1.000000,6.155778,-1.000000,104.647240,95.040788,2.955323,1,-1.000000,18.285577,100.169515,54.471371,11.239513,-1.0,30.926642,-1.000000,Vitamin B12/Folate deficiency anemia
3,7.525442,-1.000000,1.972946,2.172161,-1.000000,97.271565,-1.000000,2.320958,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,22.576325,-1.000000,No anemia
4,9.544870,411.496642,0.891182,0.000000,236.428214,104.721025,-1.000000,2.734371,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,28.634611,-1.000000,Unspecified anemia


In [5]:
class_dict = constants.CLASS_DICT
df['label'] = df['label'].replace(class_dict)
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional in

((56000, 17), (14000, 17), (56000,), (14000,))

#### The Random Agent

In [6]:
%%time
rand_agent = RandomAgent(X_test[0:], y_test[0:])
test_df = rand_agent.test()
test_df.head()

Testing done.....
Wall time: 2min 53s


,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
0,2.0,0.0,0.0,-1.0,0.0,"[ret_count, Vitamin B12/Folate deficiency anemia]",4.0,1.0
1,1.0,1.0,0.0,-1.0,0.0,[No anemia],3.0,0.0
2,2.0,2.0,0.0,-1.0,0.0,"[segmented_neutrophils, Iron deficiency anemia]",3.0,4.0
3,1.0,3.0,0.0,-1.0,0.0,[Anemia of chronic disease],4.0,3.0
4,1.0,4.0,0.0,-1.0,0.0,[Anemia of chronic disease],2.0,3.0


In [7]:
# %%time
# rand_agent.test_sample(2)

In [8]:
# from datetime import datetime
# start = datetime.now()
# rand_agent.test_sample(2)
# end = datetime.now()
# duration = end-start
# duration

In [9]:
len(X_test), len(test_df)

(14000, 14000)

In [10]:
# test_df.to_csv('../../test_dfs/many_features/random_test_df_noisy_0.4.csv', index=False)

In [11]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

12.128571428571428

In [12]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

(1.5399285714285715, -0.7627142857142857)

In [13]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(0.12128571428571429, 0.1193536303210731, 0.49936365251486314)

In [14]:
test_df.y_pred.unique()

array([1., 0., 4., 3., 2., 6., 7., 5.])